In [86]:
import numpy as np
import time
import multiprocessing
import random 
from jax import random as jrandom
import jax.numpy as jnp
import jax
from scipy.optimize import fsolve
from jax import jit, partial, grad, jacfwd
import torch 

import matplotlib.pyplot as plt
from tqdm import tqdm 

%matplotlib inline
plt.rcParams['figure.figsize'] = [16,8]
plt.style.use('ggplot')
plt.rcParams['lines.markersize'] = 8

from jax.config import config
config.update("jax_enable_x64", True)

import itertools

from utils import Logger, newton_dir_loss

ImportError: cannot import name 'newton_dir_loss'

# Look at the variance of the estimator for each function. Not noisy. 

In [88]:
def run_tests(estimator, funcs, x_0_generator, standard_deviations, dimensions, frac_samples, num_runs, seed):
    logger = Logger(["function", "dimension", "std", "frac_samples"])

    jrandom_key = jrandom.PRNGKey(seed)

    for _ in tqdm(range(num_runs)):
        
        for dim in dimensions:
            for func_name in funcs:
                F = funcs[func_name](dim)
                x_0 = x_0_generator(dim)
                true_H = F.f2(x_0.reshape(1, -1))[0]
                true_grad = F.f1(x_0.reshape(1, -1))[0]
    
                for std, frac_s in itertools.product(standard_deviations, frac_samples): 
                    num_samples = frac_s*dim

                    jrandom_key, subkey = jrandom.split(jrandom_key)
                    
                    est_H = estimator(F, x_0, std, num_samples, subkey)
                    
                    logger.add_value([func_name, dim, std, frac_s])
                    
    return logger
        

In [ ]:
B = Beta()
A = Ackley()

Q_m = 1*jnp.eye(dim)
b = jnp.zeros(dim)
Q = Quadratic(Q_m, b)

funcs = {
    "Quadratic": lambda dim: Quadratic(jnp.eye(dim), jnp.zeros(dim)),
    "Ackley": lambda dim: Ackley(),
    "Beta": lambda dim: Beta()
}

star_estimator = lambda F, x_0, std, num_samples, subkey: star_2E1(F, x_0, std, subkey, noisy_grad=False)

x_0_generator = lambda dim: jnp.ones(dim)/jnp.linalg.norm(jnp.ones(dim))

seed = 0

standard_deviations = [1, 0.1]
dimensions = [1, 10]
frac_samples = [1]
num_runs = 2


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
